# Car damage dataset preprocessor
This notebook will prepare the car damage dataset for the Peltarion platform.

Note: This notebook requires installation of Sidekick. For more information about this package, see:
https://github.com/Peltarion/sidekick

The raw dataset is available at:  https://storage.cloud.google.com/bucket-8732/car_damage/raw.zip

---


In [1]:
import os
import sidekick
import resource
import functools
import pandas as pd
from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split

## Set paths

In [2]:
# Path to the raw dataset (unzipped)
input_path = './raw'
# Path to the zip output
output_path = './preprocessed.zip'

### Get list of paths to all files

In [3]:
images_rel_path = glob(input_path + '/*/*.jpg') + glob(input_path + '/*/*.png')
print("Images found: ", len(images_rel_path))

Images found:  1538


## Create Dataframe
The class column values are derived from the names of the subfolders in the `input_path`.

The image column contains the relative path to the images in the subfolders.

### Create image and class columns

In [4]:
df = pd.DataFrame({'image': images_rel_path})
df['class'] = df['image'].apply(lambda path: os.path.basename(os.path.dirname(path)))
df.head()

,image,class
0,/Users/joakim/rep/car-damage/data/raw/door_scr...,door_scratch
1,/Users/joakim/rep/car-damage/data/raw/door_scr...,door_scratch
2,/Users/joakim/rep/car-damage/data/raw/door_scr...,door_scratch
3,/Users/joakim/rep/car-damage/data/raw/door_scr...,door_scratch
4,/Users/joakim/rep/car-damage/data/raw/door_scr...,door_scratch


## Filter images
Filter out non-RGB images

Create temporary ``image_mode`` column

In [5]:
def get_mode(path):
    im = Image.open(path)
    im.close()
    return im.mode

df['image_mode'] = df['image'].apply(lambda path: get_mode(path))
df['image_mode'].value_counts()

RGB     1512
RGBA      18
P          6
CMYK       2
Name: image_mode, dtype: int64

In [6]:
df = df[df.image_mode =='RGB']
df['image_mode'].value_counts()

RGB    1512
Name: image_mode, dtype: int64

Remove the temporary column

In [7]:
df = df.drop(['image_mode'], axis=1)

In [8]:
df['class'].value_counts()

unknown           549
door_dent         192
door_scratch      154
glass_shatter     137
tail_lamp         136
head_lamp         133
bumper_dent       129
bumper_scratch     82
Name: class, dtype: int64

## Create subsets for training and validation

In [9]:
def create_subsets(df, col='class', validation_size=0.20):
    train_data, validate_data = train_test_split(df, test_size=validation_size, random_state=42, stratify=df[[col]])
    train_data.insert(loc=2, column='subset', value='T')
    validate_data.insert(loc=2, column='subset', value='V')
    return train_data.append(validate_data, ignore_index=True)

df = create_subsets(df)
df['subset'].value_counts()

T    1209
V     303
Name: subset, dtype: int64

## Upsampling 
Upsampling (duplicating samples) can be used to prevent bias in an ubalanced dataset

In [10]:
max_size = df[df['subset']=='T']['class'].value_counts().max()
lst = [df]
for class_index, group in df[df['subset']=='T'].groupby('class'):
    lst.append(group.sample(max_size-len(group), replace=True))
df = pd.concat(lst)
df['class'].value_counts()


unknown           549
door_dent         478
door_scratch      470
tail_lamp         466
head_lamp         466
glass_shatter     466
bumper_dent       465
bumper_scratch    455
Name: class, dtype: int64

## Create dataset bundle

In [11]:
df.head()

,image,class,subset
0,/Users/joakim/rep/car-damage/data/raw/unknown/...,unknown,T
1,/Users/joakim/rep/car-damage/data/raw/head_lam...,head_lamp,T
2,/Users/joakim/rep/car-damage/data/raw/door_scr...,door_scratch,T
3,/Users/joakim/rep/car-damage/data/raw/head_lam...,head_lamp,T
4,/Users/joakim/rep/car-damage/data/raw/unknown/...,unknown,T


In [12]:
image_processor = functools.partial(sidekick.process_image, mode='crop_and_resize', size=(224, 224), file_format='jpeg')
sidekick.create_dataset(
    output_path,
    df,
    path_columns=['image'],
    preprocess={
        'image': image_processor
    }
)
# The duplicated images in the upsampled class will cause warnings

/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/1068.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/586.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/531.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/413.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/1058.jpeg'
  return 

/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/948.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/497.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/359.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/419.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/1006.jpeg'
  return s

/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/1171.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/693.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/307.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/92.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/87.jpeg'
  return sel

/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/263.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/508.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/175.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/1134.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/292.jpeg'
  return s

/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/784.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/140.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/446.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/788.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/217.jpeg'
  return se

...

/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/351.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/1144.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/681.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/682.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/856.jpeg'
  return s

/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/206.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/353.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/275.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/117.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/498.jpeg'
  return se

/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/656.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/279.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/580.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/847.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/318.jpeg'
  return se

/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/760.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/184.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/129.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/247.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/1201.jpeg'
  return s

/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/903.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/537.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/905.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/612.jpeg'
  return self._open_to_write(zinfo, force_zip64=force_zip64)
/usr/local/Cellar/python/3.7.1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/zipfile.py:1470: UserWarning: Duplicate name: 'image/95.jpeg'
  return sel